In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import json
import folium
from pyproj import Proj, transform
import mapclassify as mc
from shapely.geometry import Polygon
import geopandas as gpd
import geoplot as gplt
import geoplot.crs as gcrs

In [2]:
restaurants = pd.read_csv('restaurants.csv', encoding='cp949')
restaurants

C:\Users\lotto\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (25,27,28,29,30,36,39,41,42) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,휴업시작일자,...,공장판매직종업원수,공장생산직종업원수,건물소유구분명,보증액,월세액,다중이용업소여부,시설총규모,전통업소지정번호,전통업소주된음식,홈페이지
0,3220000,3220000-101-2023-00456,2023-03-31,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3220000,3220000-101-2023-00458,2023-03-31,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3220000,3220000-101-2023-00459,2023-03-31,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3220000,3220000-101-2023-00460,2023-03-31,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3230000,3230000-101-2023-00220,2023-03-31,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124854,3050000,3050000-101-2001-12875,20010716,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124855,3100000,3100000-101-1997-01969,19970707,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124856,3220000,3220000-101-1994-14896,19940917,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124857,3020000,3020000-101-1988-00925,19880704,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# X좌표 180000, Y좌표 435000을 (0, 0)으로 정의
# Grid 격자의 크기는 3000×3000으로 정의

x_tm = [180000 + 3000*x for x in range(0, 14)]
y_tm = [435000 + 3000*x for x in range(0, 12)]
x_grid = [x//12 for x in range(0, 168)]
y_grid = [x%12 for x in range(0,168)]
grid = pd.DataFrame([x_grid, y_grid], index=['x_grid', 'y_grid']).transpose()
grid['x_tm_start'] = [x_tm[grid.iloc[x, 0]] for x in range(0,168)]
grid['x_tm_end'] = grid['x_tm_start'] + 3000
grid['y_tm_start'] = [y_tm[grid.iloc[x, 1]] for x in range(0,168)]
grid['y_tm_end'] = grid['y_tm_start'] + 3000

grid

,x_grid,y_grid,x_tm_start,x_tm_end,y_tm_start,y_tm_end
0,0,0,180000,183000,435000,438000
1,0,1,180000,183000,438000,441000
2,0,2,180000,183000,441000,444000
3,0,3,180000,183000,444000,447000
4,0,4,180000,183000,447000,450000
...,...,...,...,...,...,...
163,13,7,219000,222000,456000,459000
164,13,8,219000,222000,459000,462000
165,13,9,219000,222000,462000,465000
166,13,10,219000,222000,465000,468000


In [4]:
restaurants_necessary = restaurants[['좌표정보(X)', '좌표정보(Y)']]
restaurants_necessary = restaurants_necessary.dropna()
restaurants_necessary['x_grid'] = (restaurants_necessary['좌표정보(X)'] -180000)//3000
restaurants_necessary['y_grid'] = (restaurants_necessary['좌표정보(Y)'] -435000)//3000
restaurants_necessary['restaurants'] = restaurants_necessary['x_grid']*12 + restaurants_necessary['y_grid']
grid_index = pd.DataFrame(restaurants_necessary['restaurants'].astype(int).value_counts())
grid = pd.merge(grid, grid_index, left_index=True, right_index=True, how='left')
grid = grid.fillna(0)

grid

,x_grid,y_grid,x_tm_start,x_tm_end,y_tm_start,y_tm_end,restaurants
0,0,0,180000,183000,435000,438000,0.0
1,0,1,180000,183000,438000,441000,0.0
2,0,2,180000,183000,441000,444000,0.0
3,0,3,180000,183000,444000,447000,0.0
4,0,4,180000,183000,447000,450000,5.0
...,...,...,...,...,...,...,...
163,13,7,219000,222000,456000,459000,0.0
164,13,8,219000,222000,459000,462000,0.0
165,13,9,219000,222000,462000,465000,0.0
166,13,10,219000,222000,465000,468000,0.0


In [5]:
proj_2097 = Proj(init='EPSG:2097')
proj_4326 = Proj(init='EPSG:4326')

start_transform = transform(proj_2097, proj_4326, grid['x_tm_start'], grid['y_tm_start'])
end_transform = transform(proj_2097, proj_4326, grid['x_tm_end'], grid['y_tm_end'])
grid_all = pd.merge(grid, pd.DataFrame(start_transform, index=['lat_start', 'lon_start']).transpose(), left_index=True, right_index=True, how='left')
grid_all = pd.merge(grid_all, pd.DataFrame(end_transform, index=['lat_end', 'lon_end']).transpose(), left_index=True, right_index=True, how='left')
grid_all

C:\Users\lotto\anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\lotto\anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\lotto\AppData\Local\Temp/ipykernel_47108/799651966.py:4: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  start_transfor

,x_grid,y_grid,x_tm_start,x_tm_end,y_tm_start,y_tm_end,restaurants,lat_start,lon_start,lat_end,lon_end
0,0,0,180000,183000,435000,438000,0.0,126.771972,37.416901,126.805792,37.443991
1,0,1,180000,183000,438000,441000,0.0,126.771891,37.443931,126.805722,37.471021
2,0,2,180000,183000,441000,444000,0.0,126.771809,37.470961,126.805653,37.498051
3,0,3,180000,183000,444000,447000,0.0,126.771727,37.497990,126.805583,37.525080
4,0,4,180000,183000,447000,450000,5.0,126.771646,37.525020,126.805514,37.552110
...,...,...,...,...,...,...,...,...,...,...,...
163,13,7,219000,222000,456000,459000,0.0,127.213071,37.606132,127.247135,37.633095
164,13,8,219000,222000,459000,462000,0.0,127.213148,37.633162,127.247225,37.660124
165,13,9,219000,222000,462000,465000,0.0,127.213226,37.660191,127.247315,37.687153
166,13,10,219000,222000,465000,468000,0.0,127.213304,37.687220,127.247406,37.714182


In [6]:
grid_list = [[[grid_all.iloc[x, 8], grid_all.iloc[x, 7]], 
             [grid_all.iloc[x, 10], grid_all.iloc[x, 7]], 
             [grid_all.iloc[x, 10], grid_all.iloc[x, 9]],
             [grid_all.iloc[x, 8], grid_all.iloc[x, 9]],
             [grid_all.iloc[x, 8], grid_all.iloc[x, 7]]] for x in range(0,168)]

polygon = []

for grid in grid_list:
    polygon.append(Polygon(grid))

grid3000 = gpd.GeoDataFrame({'geometry':polygon})
grid3000.crs = 'EPSG:4326'
grid3000['index'] = grid3000.index
grid3000['restaurants'] = grid_all['restaurants'].astype(int)
grid3000 = grid3000[grid3000['restaurants'] != 0].reset_index(drop=True)
grid3000

,geometry,index,restaurants
0,"POLYGON ((37.52502 126.77165, 37.55211 126.771...",4,5
1,"POLYGON ((37.55205 126.77156, 37.57914 126.771...",5,151
2,"POLYGON ((37.57908 126.77148, 37.60617 126.771...",6,2
3,"POLYGON ((37.47102 126.80572, 37.49810 126.805...",14,300
4,"POLYGON ((37.49805 126.80565, 37.52513 126.805...",15,367
...,...,...,...
79,"POLYGON ((37.47109 127.14486, 37.49807 127.144...",134,237
80,"POLYGON ((37.49812 127.14491, 37.52510 127.144...",135,73
81,"POLYGON ((37.52515 127.14496, 37.55213 127.144...",136,231
82,"POLYGON ((37.55218 127.14502, 37.57916 127.145...",137,339


In [7]:
grid_list = [[[grid_all.iloc[x, 7], grid_all.iloc[x, 8]], 
             [grid_all.iloc[x, 7], grid_all.iloc[x, 10]], 
             [grid_all.iloc[x, 9], grid_all.iloc[x, 10]],
             [grid_all.iloc[x, 9], grid_all.iloc[x, 8]],
             [grid_all.iloc[x, 7], grid_all.iloc[x, 8]]] for x in range(0,168)]

polygon = []

for grid in grid_list:
    polygon.append(Polygon(grid))

grid3000 = gpd.GeoDataFrame({'geometry':polygon})
grid3000.crs = 'EPSG:4326'
grid3000['index'] = grid3000.index
grid3000['restaurants'] = grid_all['restaurants'].astype(int)
grid3000 = grid3000[grid3000['restaurants'] != 0].reset_index(drop=True)
grid3000['index'] = grid3000.index
grid3000

,geometry,index,restaurants
0,"POLYGON ((126.77165 37.52502, 126.77165 37.552...",0,5
1,"POLYGON ((126.77156 37.55205, 126.77156 37.579...",1,151
2,"POLYGON ((126.77148 37.57908, 126.77148 37.606...",2,2
3,"POLYGON ((126.80572 37.47102, 126.80572 37.498...",3,300
4,"POLYGON ((126.80565 37.49805, 126.80565 37.525...",4,367
...,...,...,...
79,"POLYGON ((127.14486 37.47109, 127.14486 37.498...",79,237
80,"POLYGON ((127.14491 37.49812, 127.14491 37.525...",80,73
81,"POLYGON ((127.14496 37.52515, 127.14496 37.552...",81,231
82,"POLYGON ((127.14502 37.55218, 127.14502 37.579...",82,339


In [9]:
m = folium.Map(location=[37.5642135, 127.0016985], zoom_start=11)

sim_geo = gpd.GeoSeries(grid3000['geometry'], grid3000['index'])
geo_j = sim_geo.to_json()

geo_str = json.load(open('gu.json', encoding='utf-8'))
folium.Choropleth(
    geo_data=geo_str,
).add_to(m)

folium.Choropleth(
    geo_data=geo_j,
    fill_color='YlGnBu'
).add_to(m)

for _, r in grid3000.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo_new = gpd.GeoSeries(r['geometry'])
    geo_j_new = sim_geo_new.to_json()
    geo_j_new = folium.GeoJson(data=geo_j_new, style_function=lambda x:{'fillColor': '#FF000000', 'color': '#FF000000'})
    folium.Popup(str(r['index'])).add_to(geo_j_new)
    geo_j_new.add_to(m)

m